## Tensors
### Pytorch: Tensors

In [25]:
import torch

In [26]:
dtype = torch.float
device = torch.device("cpu")

In [27]:
# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

In [28]:
# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [29]:
# 무작위로 가중치를 초기화
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

In [31]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # 손실(loss)을 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

epoch:  50   loss: 7.969501166371629e-05
epoch:  100   loss: 4.401400656206533e-05
epoch:  150   loss: 2.7967880669166334e-05
epoch:  200   loss: 2.00710474018706e-05
epoch:  250   loss: 1.5659494238207117e-05
epoch:  300   loss: 1.2649150448851287e-05
epoch:  350   loss: 1.0716854376369156e-05
epoch:  400   loss: 9.1003939814982e-06
epoch:  450   loss: 7.997147804417182e-06
epoch:  500   loss: 7.12694100002409e-06


## Autograd
### PyTorch: Tensor와 autograd

`x`가 `x.requires_grad=True`인 Tensor라면 `x.grad`는 어떤 스칼라 값에 대한 `x`의 변화도를 갖는 또 다른 Tensor다.

In [32]:
import torch

dtype = torch.float
device = torch.device("cpu")

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

In [33]:
# 무작위의 입력과 출력 데이터를 생성
# requires_grad = False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를
# 계산할 필요가 없음을 나타냄. (requires_grad의 defalut값은 False임)
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [34]:
# 무작위로 가중치를 초기화
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를
# 계산할 필요가 있음을 나타냄.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [42]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y값을 계산.
    # 이는 Tensor를 사용한 순전파 단계와 완전히 동일하지만, 역전파 단계를 별도로
    # 구현하지 않아도 되므로, 중간값들에 대한 정보를 가지고 있을 필요가 없어서
    # 정의하지 않음
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Tensor 연산을 사용하여 손실을 계산하고 출력
    #  loss는 (1,) 형태의 Tensor이며, loss.item()은 loss의 스칼라 값
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산.
    # 이는 requires_grad=True를 갖는 모든 Tensor에 대해 손실의 변화도를 계산
    # 이후 w1.grad와 w2.grad는 w1과 w2 각각에 대한 손실의 변화도를 갖는 Tensor가 됨
    loss.backward()
    
    # 경사하강법(gradient descent)을 사용하여 가중치를 수동으로 갱신
    # torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True이지만
    # autograd에서는 이를 추적하지 않아야하기 때문
    # 다른 방법으로 weight.data 및 weight.grad.data를 조작하는 방법이있음
    # tensor.data가 tensor의 저장공간을 공유하기는 하지만, 이력을 추적하지는 않음
    # 또한, 이를 위해 torch.optim.SGD를 사용할 수 있음
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듦
        w1.grad.zero_()
        w2.grad.zero_()

epoch:  50   loss: 16779.67578125
epoch:  100   loss: 780.047119140625
epoch:  150   loss: 57.78599548339844
epoch:  200   loss: 5.3114190101623535
epoch:  250   loss: 0.5697674751281738
epoch:  300   loss: 0.06889970600605011
epoch:  350   loss: 0.009311377070844173
epoch:  400   loss: 0.0016124470857903361
epoch:  450   loss: 0.00042105757165700197
epoch:  500   loss: 0.00015764523413963616


### Pytorch: 새 autograd 함수 정의하기

내부적으로, autograd의 기본(primitive) 연산자는 실제로 Tensor를 조작하는 2개의 함수다. forward 함수는 입력 Tensor로부터 출력 Tensor를 계산하고, backward 함수는 어떤 스칼라 값에 대한 출력 Tensor의 변화도를 전달받고, 동일한 스칼라 값에 대한 입력 Tensor의 변화도를 계산한다.

Pytorch에서 `torch.autograd.Function`의 서브클래스(subclass)를 정의하고 `forward`와 `backward` 함수를 구현함으로써 사용자 정의 autograd 연산자를 손쉽게 정의할 수 있다. 그 후, 인스턴스(instance)를 생성하고 이를 함수처럼 호출하여 입력 데이터를 갖는 Tensor를 전달하는 식으로 새로운 autograd 연산자를 사용할 수 있다.

이 예제에서는 ReLU로 비선형적(nonlinearity)으로 동작하는 사용자 정의 autograd 함수를 정의하고, 2-계층 신경망에 이를 적용해보자.

In [5]:
import torch

In [6]:
class MyReLU(torch.autograd.Function):
    """
    torch.autograd.Function을 상속받아 사용자 정의 autograd Function을 구현하고,
    Tensor 연산을 하는 순전파와 역전파 단계를 구현
    """
    
    @staticmethod
    def forward(ctx, input):
        """
        순전파 단계에서는 입력을 갖는 Tensor를 받아 출력을 갖는 Tensor를 반환
        ctx는 컨텍스트 객체(context object)로 역전파 연산을 위한 정보 저장에 사용
        ctx.save_for_backward method를 사용하여 역전파 단계에서 사용할 객체를 
        저장(cache)해 둘 수 있음
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        역전파 단계에서는 출력에 대한 손실의 변화도를 갖는 Tensor를 받고, 입력에
        대한 손실의 변화도를 계산
        """
        input  = ctx.saved_tensors[0]
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [7]:
dtype = torch.float
device = torch.device("cpu")

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 무작위로 가중치를 초기화
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [8]:
learning_rate = 1e-6

for t in range(500):
    # 사용자 정의 Function을 적용하기 위해 Function.apply 메소드를 사용
    # 여기에 "relu"라는 이름을 붙이자
    relu = MyReLU.apply
    # 꼭 여기 있어야할까? iteration 밖에 있으면 안될까?

    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y 값을 계산
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # 손실을 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산.
    loss.backward()
    
    # 경사하강법(gradient descent)을 사용하여 가중치를 수동으로 갱신
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듦
        w1.grad.zero_()
        w2.grad.zero_()

epoch:  50   loss: 13888.7646484375
epoch:  100   loss: 429.4065856933594
epoch:  150   loss: 22.591934204101562
epoch:  200   loss: 1.577162504196167
epoch:  250   loss: 0.13045521080493927
epoch:  300   loss: 0.012088910676538944
epoch:  350   loss: 0.0014401051448658109
epoch:  400   loss: 0.00030147095094434917
epoch:  450   loss: 0.00010349584044888616
epoch:  500   loss: 4.843526039621793e-05


In [4]:
learning_rate = 1e-6
# 사용자 정의 Function을 적용하기 위해 Function.apply 메소드를 사용
# 여기에 "relu"라는 이름을 붙이자
relu = MyReLU.apply
# 되는 듯?
for t in range(500):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y 값을 계산
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # 손실을 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
    
    # autograd를 사용하여 역전파 단계를 계산.
    loss.backward()
    
    # 경사하강법(gradient descent)을 사용하여 가중치를 수동으로 갱신
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듦
        w1.grad.zero_()
        w2.grad.zero_()

epoch:  50   loss: 13494.4423828125
epoch:  100   loss: 578.705322265625
epoch:  150   loss: 39.98965835571289
epoch:  200   loss: 3.1663808822631836
epoch:  250   loss: 0.26957014203071594
epoch:  300   loss: 0.024226771667599678
epoch:  350   loss: 0.0025284909643232822
epoch:  400   loss: 0.0004436047456692904
epoch:  450   loss: 0.00013519717322196811
epoch:  500   loss: 5.941187555436045e-05


### TensorFlow: 정적 그래프(Static Graph)

PyTorch autograd는 Tensorflow와 많이 비슷해보입니다: 두 프레임워크 모두 연산 그래프를 정의하며, 자동 미분을 사용하여 변화도를 계산합니다. 두 프레임워크의 가장 큰 차이점은 Tensorflow의 연산 그래프는 정적 인데 반해, PyTorch는 동적 연산 그래프를 사용한다는 것입니다.

Tensorflow에서는 연산 그래프를 한 번 정의한 후 동일한 그래프를 계속해서 실행하며 가능한 다른 입력 데이터를 그래프에 전달합니다. PyTorch에서는 각각의 순전파 단계는 새로운 연산 그래프를 정의합니다.

정적 그래프는 그래프를 미리(upfront) 최적화할 수 있기 때문에 좋습니다; 예를 들어 프레임워크가 효율을 위해 일부 그래프 연산을 합치거나, 여러 GPU나 시스템(machine)에 그래프를 배포하는 전략을 제시하도록 결정할 수 있습니다. 만약 동일한 그래프를 계속 재사용하면, 초기의 값비싼 최적화 비용을 동일한 그래프를 반복하여 실행함으로써 상환(상쇄)할 수 있습니다.

정적 그래프와 동적 그래프는 제어 흐름(control flow) 측면에서도 다릅니다. 어떤 모델에서 각 데이터 지점(point)마다 다른 연산 연산을 수행하고 싶을 수 있습니다; 예를 들어 순환 신경망에서 각각의 데이터 지점마다 서로 다른 횟수만큼 펼칠(unroll) 수 있습니다; 이러한 펼침은 반복문(loop)으로 구현할 수 있습니다. 정적 그래프에서 반복문은 그래프의 일부가 돼야 합니다; 이러한 이유에서 Tensorflow는 그래프 내에 반복문을 포함하기 위해 tf.scan 과 같은 연산자를 제공합니다. 동적 그래프에서는 이러한 상황이 더 단순(Simple)해집니다: 각 예제에 대한 그래프를 즉석에서 작성하기 때문에, 일반적인 명령형(Imperative) 제어 흐름을 사용하여 각각의 입력에 따라 다른 계산을 수행할 수 있습니다.

## nn 모듈
### PyTorch: nn

연산 그래프와 autograd는 복잡한 연산자를 정의하고 도함수(derivative)를 자동으로 계산하는 매우 강력한 패러다임입니다; 하지만 규모가 큰 신경망에서는 autograd 그 자체만으로는 너무 낮은 수준(low-level)일 수 있습니다.

신경망을 구성할 때 종종 연산을 여러 계층 에 배열(arrange)하는 것으로 생각하는데, 이 중 일부는 학습 도중 최적화가 될 학습 가능한 매개변수 를 갖고 있습니다.

Tensorflow는 Keras, TensorFlow-Slim, 나 TFLearn 같은 패키지들이 연산 그래프를 더 높은 수준으로 추상화(higher-level abstraction)하여 제공하므로 신경망을 구축하는데 있어 유용합니다.

PyTorch에서는 nn 패키지가 동일한 목적으로 제공됩니다. nn 패키지는 신경망 계층(layer)들과 거의 동일한 Module 의 집합을 정의합니다. Module은 입력 Tensor를 받고 출력 Tensor를 계산하는 한편, 학습 가능한 매개변수를 갖는 Tensor 같은 내부 상태(internal state)를 갖습니다. nn 패키지는 또한 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수들도 정의하고 있습니다.


In [9]:
import torch

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in,)
y = torch.randn(N, D_out)

In [10]:
# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의
# nn.Sequential은 다른 Module들을 포함하는 Module로, 그 Module들을 순차적으로
# 적용하여 출력을 생성.
# 각각의 Linear Module은 선형 함수를 사용하여 입력으로부터 출력을 계산하고,
# 내부 Tensor에 가중치와 편향을 저장.

model = torch.nn.Sequential(
        torch.nn.Linear(D_in, H),
        torch.nn.ReLU(),
        torch.nn.Linear(H, D_out),
)

# 또한 nn 패키지에는 널리 사용되는 손실 함수들에 대한 정의도 포함되어있음
# 여기에서는 평균 제곱 오차(MSE)를 손실 함수로 사용
loss_fn = torch.nn.MSELoss(reduction='sum')

In [11]:
learning_rate = 1e-4
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산
    # Module 객체는 __call__ 연산자를 덮어써(override) 함수처럼 호출할 수 있게 함
    # 이렇게 함으로써 입력 데이터의 Tensor를 Module에 전달하여 출력 데이터의
    # Tensor를 생성
    y_pred = model(x)
    
    # 손실은 계산하고 출력한다. 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고,
    # 손실 함수는 손실 값을 갖는 Tensor를 반환한다.
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print("epoch: ", t+1, "  loss:", loss.item())
        
    # 역전파 단계를 실행하기 전에 변화도를 0으로 만든다.
    model.zero_grad()
    
    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를
    # 계산. 내부적으로 각 Module의 매개변수는 requires_grad=True 일 때 Tensor 내에 저장되므로,
    # 이 호출은 모든 모델의 모든 학습 가능한 매개변수의 변화도를 계산한다.
    loss.backward()
    
    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신. 각 매개변수는 Tensor이므로
    # 이전에 했던 것과 같이 변화도에 접근할 수 있음
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

epoch:  50   loss: 36.096343994140625
epoch:  100   loss: 2.4623265266418457
epoch:  150   loss: 0.2870693504810333
epoch:  200   loss: 0.045459549874067307
epoch:  250   loss: 0.008778390474617481
epoch:  300   loss: 0.0019468821119517088
epoch:  350   loss: 0.0004683554288931191
epoch:  400   loss: 0.00011935686052311212
epoch:  450   loss: 3.149249459966086e-05
epoch:  500   loss: 8.531626917829271e-06
